# Práctica 5
## Prototipo de sistema que identifica la matrícula de un vehículo y obtiene sus caracteres a partir de un reconocedor de texto
<ul>
<li>Nikhil Chandru Durgadas Chellaram</li>
<li>Raúl Mateus Sánchez</li>
</ul>

Para comprobar el desarrollo de la práctica: https://github.com/raulmat19/Vision-por-Computador/tree/main/Practica-5

#Paquetes necesarios

In [2]:
import cv2
import pytesseract
import glob
import os
import matplotlib.pyplot as plt
import shutil

Para el desarrollo de esta práctica, primero debemos situar el contexto de la misma. Una línea de desarrollo de esta práctica era la de emplear _YOLOv7_ como detector de vehículos, y a partir de las detecciones generadas, extraer mediante técnicas como el uso de contornos la matrícula del vehículo, si fuese posible, para procesarla de cara a que el reconocedor de texto _Tesseract_ se focalice en las zonas de probable presencia de la matrícula.

Nosotros hemos apostado por otra línea de desarrollo que puede ser fácilmente adaptada a cualquier entorno además de mantenida y actualizada. El trabajo realizado consiste en entrenar el detector _YOLOv7_ mediante un dataset creado por nosotros, configurando los parámetros pertinentes para conseguir una serie de resultados más que positivos, visualizables en [training-results](training-results/results.png) así como más detalles como la ejecución del entrenamiento en el fichero [results.txt](training-results/results.txt) y en las gráficas de la misma carpeta.

Nuestra motivación, además de contar con una GPU "tocha" para entrenar, ha sido buscar una manera de identificar matrículas aprovechando las capacidades del detector YOLOv7 al máximo y no solo para detectar vehículos, haciendo más fácil la tarea (aunque bien complicado a la hora de entrenar).

Para crear el dataset, dentro del tiempo ajustado que hemos tenido para la tarea de recopilación de imágenes, hemos empleado hasta 82 imágenes, la mayoría propias. Posteriormente, mediante la herramienta _Roboflow_, hemos subido las imágenes y realizado el _labelling_ gracias a las facilidades de esta herramienta. A continuación, _Roboflow_ permite aumentar el tamaño del dataset hasta 3 veces el número de imágenes en el plan gratuito mediante opciones como introducir imágenes del dataset en blanco y negro, con saturación, invertidas, etc. En nuestro caso, hemos aplicado las siguientes:

![dataset-details](dataset-details.png)

[Dataset en Roboflow](https://universe.roboflow.com/licenseplates-h9qfr/spanish-license-plates)<br></br>
Para acceso al mismo, contactar con nosotros (Roboflow no permite dar acceso sin un email)

Creado el dataset, queda descargarlo y ubicar las carpetas que genera (train, test y valid) dentro de la carpeta del _YOLOv7_, así como el fichero data.yaml dentro de /data. Ya solo quedaría ejecutar el comando para entrenar: 

<code>python train.py --device 0 --weights yolopt7.pt --cfg cfg/training/yolov7.yaml --data data: data/license-plate-data.yaml --hyp data/hyp.scratch.custom.yaml --workers 8 --batch-size 8 --epochs 500 --name yolov7-license-plates</code>

Para poder emplear la GPU, el comando en el GitHub de la práctica no funciona. En cambio, el siguiente sí funciona:

<code>conda install pytorch torchvision torchaudio pytorch-cuda=11.7 -c pytorch -c nvidia</code>

Una vez entrenado el detector con unos resultados aceptables (en nuestro caso, llevó más de tres intentos, aunque gracias a la GPU no se tardaba tanto), obtenemos un archivo con los mejores pesos (_weights_), el cual emplearemos a partir de ahora para detectar matrículas, llamado yolov7-license-plates.pt, el cual se encuentra en este repositorio

A partir de aquí, añadimos al repo imágenes que no se hayan usado para entrenar al detector (para no hacer trampa) y se modifica el detector, en su fichero detect.py, para que guarde como imagen las distintas detecciones en la carpeta (licenses-plates-detected) y posteriormente sean tratadas para que el reconocedor de texto pueda sacar la matrícula. El fichero detect.py también se incluye en el repositorio, cuya idea modificación proviene de: https://github.com/RizwanMunawar/yolov7-object-cropping/blob/main/detect_and_crop.py

A continuación, se define la ruta a Tesseract, se borran varios directorios para ahorrar almacenamiento y se comienza con la detección de las imágenes ubicadas en la carpeta /images/

In [3]:
# Si la ruta de Tesseract no está en el PATH, ruta al ejecutable
pytesseract.pytesseract.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract'

# Lenguajes disponibles
print(pytesseract.get_languages(config=''))
shutil.rmtree('runs/', ignore_errors=True)
shutil.rmtree('licenses-plates-detected', ignore_errors=True)

#Aplica reconocedor a imagen cargada
#print(pytesseract.image_to_string(img_rgb))

!python detect.py --weights yolov7-license-plates.pt --conf 0.75 --source images\ --save-txt --device 0

## En caso de no tener GPU o no poder usarla
# !python detect.py --weights yolov7-license-plates.pt --conf 0.75 --source images\ --save-txt --device cpu

['eng', 'equ', 'osd', 'spa', 'spa_old']
Namespace(weights=['yolov7-license-plates.pt'], source='images\\', img_size=640, conf_thres=0.75, iou_thres=0.45, device='0', view_img=False, save_txt=True, save_conf=False, nosave=False, classes=None, agnostic_nms=False, augment=False, update=False, project='runs/detect', name='exp', exist_ok=False, no_trace=False)
Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
IDetect.fuse
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 

1 License-Plate, Done. (11.1ms) Inference, (3.0ms) NMS
 The image with the result is saved in: runs\detect\exp\2465DYD.jpg
1 License-Plate, Done. (11.1ms) Inference, (1.0ms) NMS
 The image with the result is saved in: runs\detect\exp\2522LNH.jpg
1 License-Plate, Done. (12.1ms) Inference, (1.0ms) NMS
 The image with the result is saved in: runs\detect\exp\4597HXL.jpg
1 License-Plate, Done. (11.6ms) Inference, (1.0ms) NMS
 The image with the resu

YOLOR  v0.1-115-g072f76c torch 1.13.0 CUDA:0 (NVIDIA GeForce RTX 3070 Ti, 8191.5MB)

c:\Users\rauln\anaconda3\envs\yolov7\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:3191.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Model Summary: 314 layers, 36481772 parameters, 6194944 gradients, 103.2 GFLOPS
